<a href="https://colab.research.google.com/github/MindXpansion/Sentiment_Analysis-Awesome/blob/main/Sentiment_Analysis_Awesome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
import ipywidgets as widgets

# Make sure to add your API key to the Colab secrets manager under the name 'GOOGLE_API_KEY'
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Define the sentiment analysis tool configuration
sentiment_analysis_tool = {
    "function_declarations": [
        {
            "name": "sentiment_analysis",
            "description": "Classifies a short text as positive, negative, or neutral.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "The text to analyze"},
                    "sentiment": {"type": "string", "description": "The sentiment of the text (positive, negative, or neutral)"},
                    "confidence": {"type": "number", "description": "The confidence score of the sentiment analysis (0-1)"}
                },
                "required": ["text", "sentiment", "confidence"]
            }
        }
    ]
}

# Initialize the Generative Model
gemini_model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    tools=[sentiment_analysis_tool]
)

# Create a text input widget
text_input = widgets.Textarea(
    value='',
    placeholder='Enter text to analyze',
    description='Text:',
    disabled=False,
    layout={'width': '500px'}
)

# Create an output widget to display results
output_widget = widgets.Output()

# Function to handle button click
def analyze_text(button):
    with output_widget:
        output_widget.clear_output() # Clear previous output
        text_to_analyze = text_input.value
        if not text_to_analyze:
            print("Please enter some text to analyze.")
            return

        # Call the API with the sentiment analysis task configuration
        response = gemini_model.generate_content(
            f"Use the sentiment_analysis function to analyze the sentiment of the following text: {text_to_analyze}"
        )

        # Print the sentiment and confidence from the response
        import json
        try:
            sentiment_result = None
            # Iterate through the parts to find the function call
            for part in response.candidates[0].content.parts:
                if part.function_call:
                    tool_call = part.function_call
                    sentiment_result = {
                        "sentiment": tool_call.args.get('sentiment'),
                        "confidence": tool_call.args.get('confidence')
                    }
                    break # Stop after finding the first function call

            if sentiment_result:
                print(f"Sentiment: {sentiment_result['sentiment']}")
                print(f"Confidence: {sentiment_result['confidence']}")
            else:
                print("No sentiment analysis function call found in the response.")
                print(f"Raw response text: {response.text}")

        except Exception as e:
            print("Error processing response.")
            print(f"Raw response text: {response.text}")
            print(f"Error details: {e}")

# Create a button to trigger analysis
analyze_button = widgets.Button(description="Analyze Sentiment")
analyze_button.on_click(analyze_text)

# Display the widgets
display(text_input, analyze_button, output_widget)

Textarea(value='', description='Text:', layout=Layout(width='500px'), placeholder='Enter text to analyze')

Button(description='Analyze Sentiment', style=ButtonStyle())

Output()